# TensorFlow Recommenders: Quickstart

To estimate a simple matrix factorization model on the Movielens 100K dataset, the following is enough:

### Import TFRS

In [0]:
from typing import Dict, Text

import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

### Read the data

In [0]:
# Ratings data.
ratings = tfds.load('movie_lens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movie_lens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_id": tf.strings.to_number(x["movie_id"]),
    "user_id": tf.strings.to_number(x["user_id"])
})
movies = movies.map(lambda x: tf.strings.to_number(x["movie_id"]))

### Define a model:

In [0]:
class Model(tfrs.Model):

  def __init__(self):
    super().__init__()

    # Set up user representation.
    self.user_model = tf.keras.layers.Embedding(
        input_dim=2000, output_dim=64)
    # Set up movie representation.
    self.item_model = tf.keras.layers.Embedding(
        input_dim=2000, output_dim=64)
    # Set up a retrieval task and evaluation metrics over the
    # entire dataset of candidates.
    self.task = tfrs.tasks.RetrievalTask(
        corpus_metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.item_model)
        )
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.item_model(features["movie_id"])

    return self.task(user_embeddings, movie_embeddings)

### Fit and evaluate it.

In [0]:
model = Model()
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# Train.
model.fit(train.batch(4096), epochs=3)

# Evaluate.
model.evaluate(test.batch(4096), return_dict=True)

Epoch 1/3
20/20 [==============================] - 3s 174ms/step - factorized_top_k: 0.0271 - factorized_top_k/top_1_categorical_accuracy: 5.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0013 - factorized_top_k/top_10_categorical_accuracy: 0.0042 - factorized_top_k/top_50_categorical_accuracy: 0.0397 - factorized_top_k/top_100_categorical_accuracy: 0.0903 - loss: 33388.9983
Epoch 2/3
20/20 [==============================] - 4s 175ms/step - factorized_top_k: 0.0679 - factorized_top_k/top_1_categorical_accuracy: 8.7500e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0052 - factorized_top_k/top_10_categorical_accuracy: 0.0154 - factorized_top_k/top_50_categorical_accuracy: 0.1077 - factorized_top_k/top_100_categorical_accuracy: 0.2110 - loss: 31083.5711
Epoch 3/3
5/5 [==============================] - 1s 136ms/step - factorized_top_k: 0.0444 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0010 - factorized_top_k/t

{'factorized_top_k': array([0.     , 0.001  , 0.00345, 0.0617 , 0.15575], dtype=float32),
 'factorized_top_k/top_1_categorical_accuracy': 0.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.0010000000474974513,
 'factorized_top_k/top_10_categorical_accuracy': 0.003449999960139394,
 'factorized_top_k/top_50_categorical_accuracy': 0.0617000013589859,
 'factorized_top_k/top_100_categorical_accuracy': 0.15575000643730164,
 'loss': 29584.857421875}